Artifical Start

In [28]:
using LinearAlgebra
using SparseArrays
using LightGraphs
using SimpleWeightedGraphs
using GraphPlot
using BenchmarkTools
using Random
include("minFlow.jl")

checkPhase1

In [17]:
Random.seed!(3)

nodes = 5
E = Tuple{Int64,Int64}[(1,2),(1,3),(1,5),(2,3),(3,4),(4,2),(4,5),(5,3)]
cost_dict = Dict((1,2)=> 1,(1,3)=> 1, (1,5)=> 1, (2,3)=> 10, (3,4)=> 1, (4,2)=> 1, (4,5)=> 1, (5,3)=> 1)

## Artifical Start
n = nodes + 1 
b = Int64[2;5;1;-4;-4;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost
C = spzeros(n,n)
x = abs.(b)

B = ArtificalBase_table(b)
B_edgeList = ArtificalBase_list(b)
B_matrix = Base_Matrix(B_edgeList,n)
NB_edgeList = copy(E)


8-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (1, 5)
 (2, 3)
 (3, 4)
 (4, 2)
 (4, 5)
 (5, 3)

In [18]:
w = zeros(n,1)
while sum(c) != 0
    w = c'/B_matrix    
    max ,enteringEdge = reducedCost(w,NB_edgeList,C)
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = findPath(edge_u,edge_v,n,B)
    Flow_list, valid = getFlow(path,n)
    if valid
        indexes = getIndexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,n,B)
        RemoveFromTable!(edge_j,edge_i,n,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        updateBaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        c[min_index] = 0
    end
end

Restock: (6, 5)
Restock: (2, 6)
ENDING!!


In [21]:
#This will sork untill I find an exsample where more then one node is still 
# connected to artifical node
#In tandum make cost vector
cost = zeros(nodes,1)

basicEdge = Tuple{Int64,Int64}[]
for i in 1:n
    if(B_edgeList[i] != (n,n))
        if(n in B_edgeList[i])
            (B_edgeList[i][1] != n) ? push!(basicEdge, (B_edgeList[i][1],B_edgeList[i][1])) : push!(basicEdge, (B_edgeList[i][2],B_edgeList[i][2]))
            cost[i] = 0
        else
            push!(basicEdge, B_edgeList[i])
            cost[i] = cost_dict[B_edgeList[i]]
        end
    end
end

NB_edgeList = Tuple{Int64,Int64}[]
    for e in E
        if ((e in B_edgeList) == false)
            push!(NB_edgeList, e)
    end
end


B_matrix = Base_Matrix(basicEdge,nodes)
B = Base_table(basicEdge, nodes)

b = b[1:end-1] # Flow
x = x[1:end-1]

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [22]:
w = cost'/B_matrix
while sum((w*B_matrix)' .>= cost) != nodes
    
    max ,enteringEdge = reducedCost(w,NB_edgeList,C)
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = findPath(edge_u,edge_v,nodes,B)
    Flow_list, valid = getFlow(path,nodes)
    if valid
        indexes = getIndexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,nodes,B)
        RemoveFromTable!(edge_j,edge_i,nodes,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        updateBaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        cost[min_index] = cost_dict[enteringEdge]
    end
    w = cost'/B_matrix
end

In [23]:
#Check Feasibility
(w*B_matrix)' .>= cost

5×1 BitMatrix:
 1
 1
 1
 1
 1

In [24]:
x

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [25]:
cost

5×1 Matrix{Float64}:
  1.0
  0.0
  1.0
 10.0
  1.0

In [26]:
#Min Cost
cost'x

1-element Vector{Float64}:
 60.0

In [27]:
basicEdge

5-element Vector{Tuple{Int64, Int64}}:
 (1, 5)
 (2, 2)
 (3, 4)
 (2, 3)
 (4, 5)